## Pipeline SVM: Preprocesamiento, Búsqueda de Hiperparámetros y Evaluación

En las siguientes celdas se carga el dataset limpio, se define un pipeline que incluye preprocesamiento (escalado) y un clasificador SVM, se realiza búsqueda sistemática de hiperparámetros con `GridSearchCV`, y se evalúa el mejor modelo en un set de prueba. Se guardan resultados resumidos para extraer conclusiones.

In [ ]:
# SVM pipeline: carga datos, preprocesado, GridSearch y evaluación
import json
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv('../data/diabetes_012_cleaned.csv') 

with open('../data/dictionary.json', 'r') as f:
    data_dict = json.load(f)

# Rutas
DATA_PATH = Path('..') / 'data' / 'diabetes_012_processed.csv'
DICT_PATH = Path('..') / 'data' / 'dictionary.json'
OUT_DIR = Path('.') / 'svm_results'
OUT_DIR.mkdir(exist_ok=True)

In [3]:
def balance_dataset_undersampling_smote(df, target_col='Diabetes', random_state=42):
    """
    Balancear dataset combinando undersampling y SMOTE:
    - Clase 0 (No diabético): undersampling - reducir hasta ~1.5x de la clase mayoritaria entre 1 y 2
    - Clase 1 (Pre-diabetes): SMOTE - generar sintéticas hasta alcanzar ~90% de la clase 2 (Diabetes)
    - Clase 2 (Diabetes): mantener sin cambios (referencia)
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame con la columna target
    target_col : str
        Nombre de la columna objetivo (default: 'Diabetes')
    random_state : int
        Para reproducibilidad
        
    Returns:
    --------
    pd.DataFrame
        DataFrame balanceado
    """
    from imblearn.over_sampling import SMOTE
    import pandas as pd
    
    print("=" * 70)
    print("BALANCEO DE DATASET: UNDERSAMPLING + SMOTE")
    print("=" * 70)
    
    # Separar features y target
    X = df.drop(columns=[target_col])
    y = df[target_col]
    
    # Contar clases
    print(f"\n📊 Distribución inicial:")
    class_counts = y.value_counts().sort_index()
    for cls, count in class_counts.items():
        pct = 100 * count / len(y)
        print(f"  Clase {cls}: {count:,} muestras ({pct:.2f}%)")
    
    class_2_count = class_counts.get(2, 0)
    target_count = int(class_2_count * 0.9)  # 90% de la clase 2
    
    print(f"\n🎯 Objetivo final:")
    print(f"  - Clase 0: {target_count:,} muestras (undersampling)")
    print(f"  - Clase 1: {target_count:,} muestras (SMOTE)")
    print(f"  - Clase 2: {class_2_count:,} muestras (sin cambios)")
    
    # Paso 1: Undersampling para clase 0
    print(f"\n[1/2] Aplicando undersampling a clase 0...")
    X_0 = X[y == 0]
    y_0 = y[y == 0]
    
    # Muestrear aleatoriamente
    sample_idx = np.random.RandomState(random_state).choice(
        len(X_0), size=target_count, replace=False
    )
    X_0_sampled = X_0.iloc[sample_idx]
    y_0_sampled = y_0.iloc[sample_idx]
    print(f"✓ Clase 0 reducida: {len(y_0)} → {len(y_0_sampled):,} muestras")
    
    # Paso 2: SMOTE para clase 1
    print(f"\n[2/2] Aplicando SMOTE a clase 1...")
    
    # Combinar clase 0 reducida con clase 1 original
    X_temp = pd.concat([X_0_sampled, X[y == 1]])
    y_temp = pd.concat([y_0_sampled, y[y == 1]])
    
    # Aplicar SMOTE solo a clase 1 (generar hasta target_count)
    sampling_strategy = {
        1: target_count  # Generar sintéticas para clase 1 hasta este número
    }
    
    smote = SMOTE(sampling_strategy=sampling_strategy, random_state=random_state, k_neighbors=5)
    X_balanced, y_balanced = smote.fit_resample(X_temp, y_temp)
    
    count_1_before = (y_temp == 1).sum()
    count_1_after = (y_balanced == 1).sum()
    print(f"✓ Clase 1 expandida: {count_1_before} → {count_1_after:,} muestras (SMOTE sintéticas: {count_1_after - count_1_before:,})")
    
    # Paso 3: Añadir clase 2 sin cambios
    print(f"\n[3/3] Añadiendo clase 2 (sin cambios)...")
    X_balanced = pd.concat([X_balanced, X[y == 2]])
    y_balanced = pd.concat([y_balanced, y[y == 2]])
    print(f"✓ Clase 2 mantenida: {class_2_count:,} muestras")
    
    # Combinar en DataFrame
    df_balanced = X_balanced.copy()
    df_balanced[target_col] = y_balanced.values
    df_balanced = df_balanced.reset_index(drop=True)
    
    # Mostrar resultados
    print(f"\n" + "=" * 70)
    print("📈 Distribución final:")
    print("=" * 70)
    final_counts = df_balanced[target_col].value_counts().sort_index()
    total = len(df_balanced)
    for cls, count in final_counts.items():
        pct = 100 * count / total
        print(f"  Clase {cls}: {count:,} muestras ({pct:.2f}%)")
    
    print(f"\n✅ Dataset balanceado: {len(df)} → {len(df_balanced):,} muestras")
    print(f"   Incremento: {len(df_balanced) - len(df):,} muestras nuevas (sintéticas + reducidas)")
    
    return df_balanced


In [ ]:
# print("\n" + "=" * 70)
# print("BALANCEANDO DATASET...")
# print("=" * 70)
# df_balanced = balance_dataset_undersampling_smote(df, target_col='Diabetes', random_state=42)

# df_balanced.to_csv('../data/diabetes_012_processed.csv', index=False)


BALANCEANDO DATASET...
BALANCEO DE DATASET: UNDERSAMPLING + SMOTE

📊 Distribución inicial:
  Clase 0: 213,703 muestras (84.24%)
  Clase 1: 4,631 muestras (1.83%)
  Clase 2: 35,346 muestras (13.93%)

🎯 Objetivo final:
  - Clase 0: 31,811 muestras (undersampling)
  - Clase 1: 31,811 muestras (SMOTE)
  - Clase 2: 35,346 muestras (sin cambios)

[1/2] Aplicando undersampling a clase 0...
✓ Clase 0 reducida: 213703 → 31,811 muestras

[2/2] Aplicando SMOTE a clase 1...
✓ Clase 1 expandida: 4631 → 31,811 muestras (SMOTE sintéticas: 27,180)

[3/3] Añadiendo clase 2 (sin cambios)...
✓ Clase 2 mantenida: 35,346 muestras

📈 Distribución final:
  Clase 0: 31,811 muestras (32.14%)
  Clase 1: 31,811 muestras (32.14%)
  Clase 2: 35,346 muestras (35.71%)

✅ Dataset balanceado: 253680 → 98,968 muestras
   Incremento: -154,712 muestras nuevas (sintéticas + reducidas)


In [ ]:
# Cargar datos
df = pd.read_csv(DATA_PATH)
with open(DICT_PATH, 'r', encoding='utf-8') as f:
    data_dict = json.load(f)

print(f"✓ Datos cargados: {df.shape[0]} muestras, {df.shape[1]} columnas\n")



[1/5] Cargando datos: 100%|██████████| 1/1 [00:00<00:00,  1.68it/s]

✓ Datos cargados: 98968 muestras, 22 columnas



In [6]:
X = df.drop(columns=['Diabetes'])
y = df['Diabetes']

ordinal_vars = [k for k, v in data_dict.items() if v.get('type') == 'ordinal' and k in X.columns]
categorical_vars = [k for k, v in data_dict.items() if v.get('type') == 'categorical' and k in X.columns]
numerical_vars = [k for k, v in data_dict.items() if v.get('type') == 'numerical' and k in X.columns]

num_features = numerical_vars + ordinal_vars
cat_features = categorical_vars

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
], remainder='drop')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [8]:
pipe = Pipeline([
    ('preproc', preprocessor),
    ('clf', SVC(probability=True))
])

param_grid = {
    # 'clf__C': [0.1, 1, 10, 100],
    # 'clf__kernel': ['linear', 'rbf'],
    # 'clf__gamma': ['scale', 'auto']
    'clf__C': [1, 10],
    'clf__kernel': ['linear'],
    'clf__gamma': ['scale']
}

n_combinations = len(param_grid['clf__C']) * len(param_grid['clf__kernel']) * len(param_grid['clf__gamma'])
cv_folds = 5
total_fits = n_combinations * cv_folds

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=cv_folds,
    scoring='f1_macro'
    # n_jobs=1
    # verbose=0
)




In [ ]:
grid.fit(X_train, y_train)

# Resultados
best = grid.best_estimator_

⏳ Entrenando:   0%|                                    | 0/10 [00:00<?, ?fit/s]

In [ ]:
print(f"\n✓ Mejores hiperparámetros encontrados:")
for param, value in grid.best_params_.items():
    print(f"  - {param}: {value}")
print(f"\n✓ Mejor score CV (f1_macro): {grid.best_score_:.4f}")

In [ ]:
# Guardar CV results
print("\n[5/5] Guardando resultados...")
cv_results = pd.DataFrame(grid.cv_results_)
cv_results.to_csv(OUT_DIR / 'gridcv_results.csv', index=False)

# Evaluación en test set
y_pred = grid.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Test Set (SVM)')
plt.tight_layout()
plt.savefig(OUT_DIR / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
print("\n" + "=" * 70)
print("MÉTRICAS EN CONJUNTO DE PRUEBA (TEST SET)")
print("=" * 70)
print(f"\n✓ Accuracy: {acc:.4f}")
print(f"✓ F1-Score (macro): {f1:.4f}")
print(f"\nReporte detallado por clase:")
print(classification_report(y_test, y_pred))


# Guardar resumen de métricas
summary = {
    'best_params': grid.best_params_,
    'cv_best_score_f1_macro': float(grid.best_score_),
    'test_accuracy': float(acc),
    'test_f1_macro': float(f1)
    }

with open(OUT_DIR / 'summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)